In [1]:
!pip install torch torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cu102.html

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu102.html


In [2]:
from statistics import mode
import torch

import torch_geometric
from datetime import datetime
from torch_geometric.nn import GAE
import torch_geometric.transforms as T

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from time import time
from datetime import datetime

from actor import PtrNet1
from critic import PtrNet2
from env import Env_tsp
from config import Config, load_pkl, pkl_parser
from data import Generator
from torch_geometric.transforms import RandomLinkSplit

# torch.autograd.set_detect_anomaly(True)
torch.backends.cudnn.benchmark = True


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
print(device)

In [4]:
def generate_embedding(batch_coord):
  res_arr = []

  #loop through batch to generate batch of embeddings
  for coord in batch_coord:

    
    coord = coord.float()
    perm = itertools.permutations(range(10), 2)
    l1 = []
    l2 = []

    for x in perm:
        l1.append(x[0])
        l2.append(x[1])

    a1 = torch.LongTensor(l1)
    a2 = torch.LongTensor(l2)

    res = [a1,a2]
    res = torch.stack(res)

    
    #feat is x.data
    transform = T.Cartesian(cat=False)
    data = Data(x=coord, edge_index=torch.LongTensor(res), pos=coord)
    data = transform(data) #apply cartesian transform to add edge_attributes as distance between nodes
    data = RandomLinkSplit(data)
    data = data.num_val
    #generate embedding for graph nodes
    embedding = model_embed.encode(data.x, data.edge_index, data.edge_attr)
    res_arr.append(embedding)
  #print(torch.stack([res_arr[0], res_arr[1]], dim=0))

  #from list of graph embeddings reshape into batch of embeddings
  batch = torch.stack([res_arr[0], res_arr[1]], dim=0)
  #print(f"shape of batch is {batch.shape}")



  return batch

    

In [ ]:




def train_model(cfg, env, log_path = None):
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	
	date = datetime.now().strftime('%m%d_%H_%M')
	if cfg.islogger:
		param_path = cfg.log_dir + '%s_%s_param.csv'%(date, cfg.task)# cfg.log_dir = ./Csv/
		print(f'generate {param_path}')
		with open(param_path, 'w') as f:
			f.write(''.join('%s,%s\n'%item for item in vars(cfg).items()))

	#define actor model
	act_model = PtrNet1(cfg)
	
    
	if cfg.optim == 'Adam':
		act_optim = optim.Adam(act_model.parameters(), lr = cfg.lr)
	if cfg.is_lr_decay:
		act_lr_scheduler = optim.lr_scheduler.StepLR(act_optim, 
						step_size=cfg.lr_decay_step, gamma=cfg.lr_decay)
	device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
	act_model = act_model.to(device)

	if cfg.mode == 'train':
		cri_model = PtrNet2(cfg)
		if cfg.optim == 'Adam':
			cri_optim = optim.Adam(cri_model.parameters(), lr = cfg.lr)
		if cfg.is_lr_decay:
			cri_lr_scheduler = optim.lr_scheduler.StepLR(cri_optim, 
						step_size = cfg.lr_decay_step, gamma = cfg.lr_decay)
		cri_model = cri_model.to(device)
		ave_cri_loss = 0.

	mse_loss = nn.MSELoss()
	dataset = Generator(cfg, env)
	dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

	ave_act_loss, ave_L = 0., 0.
	min_L, cnt = 1e7, 0
	t1 = time()
	

	# for i, inputs in tqdm(enumerate(dataloader)):
	for i, inputs in enumerate(dataloader):
		
		inputs = inputs.to(device)
		#extract first set of nodes from batch to plot a graph for every epoch
		coord = inputs[0]
		embedding = generate_embedding(inputs)

		


- Here the actor model is fed with the embeddings

In [ ]:
		
		pred_tour, ll = act_model(embedding, device)
		tour = pred_tour[0]
		env.show(coord, tour)
		real_l = env.stack_l_fast(embedding, pred_tour)
		if cfg.mode == 'train':
			pred_l = cri_model(embedding, device)
			cri_loss = mse_loss(pred_l, real_l.detach())
			cri_optim.zero_grad()
			cri_loss.backward(retain_graph=True)
			nn.utils.clip_grad_norm_(cri_model.parameters(), max_norm = 1., norm_type = 2)
			cri_optim.step()
			if cfg.is_lr_decay:
				cri_lr_scheduler.step()
		elif cfg.mode == 'train_emv':
			if i == 0:
				L = real_l.detach().mean()
			else:
				L = (L * 0.9) + (0.1 * real_l.detach().mean())
			pred_l = L

		adv = real_l.detach() - pred_l.detach()
		act_loss = (adv * ll).mean()
		act_optim.zero_grad()
		act_loss.backward(retain_graph=True)
		nn.utils.clip_grad_norm_(act_model.parameters(), max_norm = 1., norm_type = 2)
		act_optim.step()
		if cfg.is_lr_decay:
			act_lr_scheduler.step()

		ave_act_loss += act_loss.item()
		if cfg.mode == 'train':
			ave_cri_loss += cri_loss.item()
		ave_L += real_l.mean().item()
		
		if i % cfg.log_step == 0:
			t2 = time()
			if cfg.mode == 'train':	
				print('step:%d/%d, actic loss:%1.3f, critic loss:%1.3f, L:%1.3f, %dmin%dsec'%(i, cfg.steps, ave_act_loss/(i+1), ave_cri_loss/(i+1), ave_L/(i+1), (t2-t1)//60, (t2-t1)%60))
				if cfg.islogger:
					if log_path is None:
						log_path = cfg.log_dir + '%s_%s_train.csv'%(date, cfg.task)#cfg.log_dir = ./Csv/
						with open(log_path, 'w') as f:
							f.write('step,actic loss,critic loss,average distance,time\n')
					else:
						with open(log_path, 'a') as f:
							f.write('%d,%1.4f,%1.4f,%1.4f,%dmin%dsec\n'%(i, ave_act_loss/(i+1), ave_cri_loss/(i+1), ave_L/(i+1), (t2-t1)//60, (t2-t1)%60))
			
			if(ave_L/(i+1) < min_L):
				min_L = ave_L/(i+1)	
			else:
				cnt += 1
				print(f'cnt: {cnt}/20')
				if(cnt >= 500):
					print('early stop, average cost cant decrease anymore')
					if log_path is not None:
						with open(log_path, 'a') as f:
							f.write('\nearly stop')
					break
			t1 = time()
	if cfg.issaver:		
		torch.save(act_model.state_dict(), cfg.model_dir + '%s_%s_step%d_act.pt'%(cfg.task, date, i))#'cfg.model_dir = ./Pt/'
		print('save model...')
		


In [ ]:
        
        
from torch_geometric.nn import GAE
import itertools
from torch_geometric.data import Data
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



In [6]:
from torch_geometric.nn import SplineConv

class GraphEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphEncoder, self).__init__()
        self.conv1 = SplineConv(in_channels, 2 * out_channels, dim=2, kernel_size=3 ) # cached only for transductive learning
        self.conv2 = SplineConv(2 * out_channels, out_channels,dim=2, kernel_size=3) # cached only for transductive learning

    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr).relu()
        return self.conv2(x, edge_index, edge_attr)
    

In [7]:
num_features = 2
out_channels = 16

model_embed = GAE(GraphEncoder(2,16))
model_embed.load_state_dict(torch.load("./models/model_SplineConv_20000.pt"))

model_embed = model_embed.to(device)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__ == '__main__':
	cfg = Config()
	env = Env_tsp(cfg)

	train_model(cfg, env)


## Partial conclusions as of 27/04
- Despite the loss values both for the actor and the critic model going down around 0.3~0.4, and the average length (for the 10 nodes example) going down to 2 (which is a more than optimal value), the tours given by the model are not satisfactory.
- I'm afraid that the average length is so low because in the thousands of tours that it performs (from environments generated randomly) , there are some where the distance between points is almost none so it brings the average down to an 'optimal value'.
- I'll tweak out the environment to force a certain distance between nodes, otherwise I currently have no other explanation for this behaviour.